# Super Inheritance by Hettinger
https://rhettinger.wordpress.com/2011/05/26/super-considered-super/

Basic use case, a sublcass for extending a method from on of the builtin classes:

In [2]:
class LoggingDict(dict):
    def __setitem__(self, key, value):
        logging.info('Setting %r to %r' % (key, value))
        super().__setitem__(key,value)

This class has all the same capabilities as its parent, dict, but it extends the _setitem_ method to make log entries whenever a key is updated. After making a log entry, the method uses super() to delegate the work for actually updating the dictionary with the key/value pair.

Before super() was introduced, we would have hardwired the call with dict._setitem_(self, key, value). However, super() is better because it is a computed indirect reference (вычисленная косвенная ссылка).

One benefit of indirection is that we don’t have to specify the delegate class by name. If you edit the source code to switch the base class to some other mapping, the super() reference will automatically follow. You have a single source of truth:

In [3]:
# class LoggingDict(SomeOtherMapping):
#     def __setitem__(self, key, value):
#         logging.info('Setting %r to %r' % (key, value))
#         super().__setitem__(key,value)

In addition to isolating changes, there is another major benefit to computed indirection, one that may not be familiar to people coming from static languages. Since the indirection is computed at runtime, we have the freedom to influence the calculation so that the indirection will point to some other class.

The calculation depends on both the class where super is called and on the instance’s tree of ancestors. The first component, the class where super is called, is determined by the source code for that class. In our example, super() is called in the LoggingDict._setitem_ method. That component is fixed. The second and more interesting component is variable (we can create new subclasses with a rich tree of ancestors).

Let’s use this to our advantage to construct a logging ordered dictionary without modifying our existing classes:

In [4]:
import collections

class LoggingOD(LoggingDict, collections.OrderedDict):
    pass

LoggingOD.__mro__

(__main__.LoggingOD,
 __main__.LoggingDict,
 collections.OrderedDict,
 dict,
 object)

The ancestor tree for our new class is: LoggingOD, LoggingDict, OrderedDict, dict, object. For our purposes, the important result is that OrderedDict was inserted after LoggingDict and before dict! This means that the super() call in LoggingDict._setitem_ now dispatches the key/value update to OrderedDict instead of dict.

Think about that for a moment. We did not alter the source code for LoggingDict. Instead we built a subclass whose only logic is to compose two existing classes and control their search order.

The search order or the ancestor tree is officially known as the **Method Resolution Order (MRO)**

The Sequence includes the class, its base classes, and the base classes of those bases and so on until reaching *object* which is the root class of all classes. The sequence is ordered so that a class always appears before its parents, and if there are multiple parents, they keep the same order as the tuple of base classes. 



In [5]:
LoggingOD.__bases__

(__main__.LoggingDict, collections.OrderedDict)

The MRO of LoggingOD is the one order that follows from those constraints:

- LoggingOD precedes its parents, LoggingDict and OrderedDict
- LoggingDict precedes OrderedDict because LoggingOD._bases_ is (0-main_.LoggingDict, collections.OrderedDict). **_ bases _** is a tuple containing the base classes, in the order of their occurrence in the base class list
- LoggingDict precedes its parent which is dict
- OrderedDict preceds its parent which is dict
- dict precedes its parent which is object

The process of solving those constraints is known as linearization. There are a number of good papers on the subject, but to create subclasses with an MRO to our liking, we only need to know the two constraints: children precede their parents and the order of appearance in __ bases __ is respected. 

**Practical Advice**
super() is in the business of delegating method calls to some class in the instance's ancestor tree. For reordable method calls to work, the classes need to be designed cooperatively. This presents three easily solved practical issues:
- the method being called by super() needs to exist
- the caller and callee need to have a matching argument signature
- and every occurence of the method needs to use super()

**Strategies for getting the caller's arguments to match the signature of the called method:**

This is a little more challenging than traditional method calls where the callee is known in advance. With super(), the callee is not known at the time a class is written (because a subclass written later may introduce new classes into the MRO).

One approach is to stick with a fixed signature using positional arguments. This works well with methods like _setitem_ which have a fixed signature of two arguments, a key and a value. This technique is shown in the LoggingDict example where _setitem_ has the same signature in both LoggingDict and dict.

A more flexible approach is to have every method in the ancestor tree cooperatively designed to accept keyword arguments and a keyword-arguments dictionary, to remove any arguments that it needs, and to forward the remaining arguments using **kwds, eventually leaving the dictionary empty for the final call in the chain.

Each level strips-off the keyword arguments that it needs so that the final empty dict can be sent to a method that expects no arguments at all (for example, object._init_ expects zero arguments):

In [9]:
class Shape:
    def __init__(self, shapename, **kwds):
        self.shapename = shapename
        super().__init__(**kwds)
        
class ColoredShape(Shape):
    def __init__(self, color, **kwds):
        self.color = color
        super().__init__(**kwds)
        
cs = ColoredShape(color='red', shapename='circle')

**How make sure the target method exists?**

The above example shows the simplest case. We know that object has an _init_ method and that object is always the last class in the MRO chain, so any sequence of calls to super()._ init _ is guaranteed to end with a call to object._init_ method. In other words, we’re guaranteed that the target of the super() call is guaranteed to exist and won’t fail with an AttributeError.

For cases where object doesn’t have the method of interest (a draw() method for example), we need to write a root class that is guaranteed to be called before object. The responsibility of the root class is simply to eat the method call without making a forwarding call using super().

Root.draw can also employ defensive programming using an assertion to ensure it isn’t masking some other draw() method later in the chain.  This could happen if a subclass erroneously incorporates a class that has a draw() method but doesn’t inherit from Root.:

In [13]:
class Root:
    def draw(self):
        print('Delegetion chain stops here')
        #the delegation chain stops here
        assert not hasattr(super(), 'draw')
        
class Shape(Root):
    def __init__(self, shapename, **kwds):
        self.shapename = shapename
        super().__init__(**kwds)
        
    def draw(self):
        print('Drawing. Setting shape to:', self.shapename)
        super().draw()
        
class ColoredShape(Shape):
    def __init__(self, color, **kwds):
        self.color = color
        super().__init__(**kwds)
    def draw(self):
        print('Drawing. Setting color to:', self.color)
        super().draw()
    
cs = ColoredShape(color='blue', shapename='square')
cs.draw()

Drawing. Setting color to: blue
Drawing. Setting shape to: square
Delegetion chain stops here


In [14]:
ColoredShape.__mro__

(__main__.ColoredShape, __main__.Shape, __main__.Root, object)

If subclasses want to inject other classes into the MRO, those other classes also need to inherit from Root so that no path for calling draw() can reach object without having been stopped by Root.draw. This should be clearly documented so that someone writing new cooperating classes will know to subclass from Root. This restriction is not much different than Python’s own requirement that all new exceptions must inherit from BaseException.

The techniques shown above assure that super() calls a method that is known to exist and that the signature will be correct; however, we’re still relying on super() being called at each step so that the chain of delegation continues unbroken. This is easy to achieve if we’re designing the classes cooperatively – just add a super() call to every method in the chain.

The three techniques listed above provide the means to design cooperative classes that can be composed or reordered by subclasses.

**How to Incorporate a Non-cooperative Class** 

Occasionally, a subclass may want to use cooperative multiple inheritance techniques with a third-party class that wasn’t designed for it (perhaps its method of interest doesn’t use super() or perhaps the class doesn’t inherit from the root class). This situation is easily remedied by creating an *adapter class* that plays by the rules.

For example, the following Moveable class does not make super() calls, and it has an _init_() signature that is incompatible with object._init_, and it does not inherit from Root:

In [16]:
class Moveable:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def draw(self):
        print('Drawing at position:', self.x, self.y)

If we want to use this class with our cooperatively designed ColoredShape hierarchy, we need to make an adapter with the requisite super() calls:

In [17]:
class MoveableAdapter(Root):
    def __init__(self, x, y, **kwds):
        self.movable = Moveable(x, y)
        super().__init__(**kwds)
    def draw(self):
        self.movable.draw()
        super().draw()

class MovableColoredShape(ColoredShape, MoveableAdapter):
    pass

MovableColoredShape(color='red', shapename='triangle',
                    x=10, y=20).draw()

Drawing. Setting color to: red
Drawing. Setting shape to: triangle
Drawing at position: 10 20
Delegetion chain stops here
